In [1]:
import time
import os
import json
import sys
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
zipsTop500 = [
    94123, 19104, 77494,90011, 75034,60629,11211,
    77449,75070,79936,11368,77084,11385,90650,8701,91331,11208,78660,10467,90201,90044,10473,92507,11216,92324,
    92335,37013,11236,92336,94565,11373,11226,77479,90250,90805,90280,60618,91342,77433,75052,30044,92503,10025,77584,
    29483,77573,11207,92683,11220,77429,91710,10456,11214,11234,10314,92154,78521,926,91911,60639,30043,92804,11219,
    11206,11230,75217,92376,60632,93307,92592,92704,10458,93722,11746,33027,11435,78520,78586,93117,95355,23320,
    60647,11221,94112,93033,93727,95076,78613,92345,91744,79938,92126,91709,66062,77379,92509,60617,22193,11377,11229,
    78666,99301,60804,11355,27610,91335,95823,78542,37042,30024,11223,75035,78572,60608,79912,11233,78130,95630,725,
    28269,10453,60625,76244,10468,47906,78577,94533,78641,94544,77083,75211,60623,87121,23464,11203,85142,11235,77459,
    76063,91706,78245,90706,96706,92105,11204,77450,33311,33024, 94080,84015,75067,19111,95624,94558,43123,7305,30004,
    30349,93550,60634,93257,10462,95035,75228,89108,37211,93274,94587,63376,93535,33023,91730,34787,92882,10457,10029,
    33012,92805,33411,94536,10466,90255,10452,10002,93306,80013,19120,97701,91910,97006,90003,92553,33647,11212,77095,
    75002,84095,85364,27587,11218,85032,33025,22191,30040,94110,34953,11375,92880,10469,76028,60614,45011,73099,98052,
    93065,97007,19124,80219,30041,78046,30127,60505,97229,92563,60085,91766,92021,84118,80134,953,89110,91402,96797,
    60641,30281,98012,95123,11215,32828,92114,7055,85204,85225,87114,60640,20011,75068,30052,33186,90262,94509,85281,
    85301,77036,20002,30022,95687,76179,89031,78596,93230,60620,77845,90631,30047,21234,93536,30096,28027,95747,11691,
    11209,37075,75243,83646,7087,10472,78045,23462,33015,92407,30135,90026,11706,84043,11213,90037,48197,94538,33157,
    92703,89115,89121,94541,99654,90022,30331,95758,78251,32218,85308,20019,92801,66061,95206,43081,92677,94513,91343,
    23322,94015,92562,7002,17603,77072,60628,92201,38401,29681,11201,10027,10977,85041,94501,87120,85033,28215,11432,
    84404,14850,29072,33463,7093,46307,32822,28078,2301,30062,20147,30188,91762,32765,77346,98682,92115,8753,93906,
    60651,78664,94806,20878,95020,44256,43026,85326,40475,92346,78745, 60657,20906,10463,80015,6902,28277,19720,10024,
    91977,92404,95127,44035,91770,11372,85008,89123,78574,78250,33313,19143,11717,90640,90019,91950,75071,27406,60402,
    75043,42101,92707,11434,90660,32210,92627,92504,11210,77373,79928,95608,77396,10023,43055,43130,78640,78240,77077,
    75150,60073,74012,80634,78249,93905,19134,75040,95014,92647,91702,78228,77407,60619,10701,94122,30058,73160,32244,
    34711,95111,78504,7047,37167,48180,78254,10312,90004,91732,53215,32818,37122,21740,91761,90731,92630,2360,30263,
    2148,77521,75227,34744, 29445,95051,93312,92057,28314,95828,33458,72401,30144,30083,33511,33142,77840,75056,
    956,60609,2155,37129,90042,33414,32808,83709,29485,92570,33125,78501,10031,37064,20874,32825,77386,22030,30906,
    77375,72712,85122,6010,37128,78753,76137,95112,33064,96818,44060,43230,93030,79924,22407,10032,76010,92571,
    22192,21122,21117,60016
]

zipsShortList = zipsTop500[0:8]

# http://shopping.google.com/

In [3]:
company_names = [
    'ulla+johnson',
    'mark+cross',
    'le+specs',
    'jl+audio',
    'obey+clothing',
    'roc+skin+care',
    'hurley'
]

ulla = ['ulla+johnson']

url_base = 'https://www.google.com/search?q=*&tbm=shop'

In [4]:
chrome_options = Options()
chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")

browser = webdriver.Chrome(options=chrome_options)

In [5]:
def get_storeInfo(store):
    data = {}
    data['name'] = store.find_element(By.CLASS_NAME, 'MxVeme').text
    data['google_maps_link'] = store.find_element(By.CLASS_NAME, 'k7eIUb').find_element(By.TAG_NAME, 'a').get_attribute('data-url')
    data['address'] = store.find_element(By.CLASS_NAME, 'lSS0Af').text

    return data

In [6]:
resultsList = []

for zipcode in zipsShortList:
    for company in ulla:
        current_combo = url_base.replace("*", company+'+'+str(zipcode))
        print(current_combo)
        browser.get(current_combo) #Get the link
        morePlaces = True
        
        results = browser.find_element(By.XPATH, '//div[@jscontroller="lcX38e"]')
        stores = results.find_elements(By.CLASS_NAME, 'FFnM0')
        print("LEN:", len(stores))
        
        while morePlaces == True:
            #for length in range(len(stores)-3): #How many times to click the more places button
            try:
                button = results.find_element(By.CLASS_NAME, 't6JUTe')
                button.click()
                time.sleep(1)
            except:
                pass
                print("no more 'more places' button")
                morePlaces = False
        
        for store in stores:
            resultsList.append(get_storeInfo(store))
            
        #time.sleep(1)
        
        browser.quit()
        browser = webdriver.Chrome(options=chrome_options)

https://www.google.com/search?q=ulla+johnson+94123&tbm=shop
LEN: 18
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+19104&tbm=shop
LEN: 13
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+77494&tbm=shop
LEN: 23
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+90011&tbm=shop
LEN: 22
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+75034&tbm=shop
LEN: 28
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+60629&tbm=shop
LEN: 19
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+11211&tbm=shop
LEN: 16
no more 'more places' button
https://www.google.com/search?q=ulla+johnson+77449&tbm=shop
LEN: 22
no more 'more places' button


In [8]:
df = pd.DataFrame(resultsList)
df

,name,google_maps_link,address
0,Neiman Marcus,https://maps.google.com/maps?daddr=150+Stockto...,"150 Stockton Street, San Francisco"
1,dress,https://maps.google.com/maps?daddr=2271+Chestn...,"2271 Chestnut Street, San Francisco"
2,Nordstrom,https://maps.google.com/maps?daddr=1200+Broadw...,"1200 Broadway Plaza, Walnut Creek"
3,Neiman Marcus,https://maps.google.com/maps?daddr=400+Stanfor...,"400 Stanford Shopping Center, Palo Alto"
4,Nordstrom,https://maps.google.com/maps?daddr=550+Stanfor...,"550 Stanford Shopping Center, Palo Alto"
...,...,...,...
156,Ann Taylor,https://maps.google.com/maps?daddr=12850+Memor...,"12850 Memorial Drive, Houston"
157,LOFT,https://maps.google.com/maps?daddr=2400+Univer...,"2400 University Blvd, Houston"
158,LOFT,https://maps.google.com/maps?daddr=1201+Lake+W...,"1201 Lake Woodland Drive, The Woodlands"
159,LOFT,https://maps.google.com/maps?daddr=303+Memoria...,"303 Memorial City, Houston·"
